# Stage 11 — Evaluation, Risk & Communication

**Chain statement:** In the lecture, we learned to bootstrap uncertainty, compare scenarios, and communicate risks. Here we adapt that workflow to produce a validated baseline and stakeholder-facing summary.

## 0) Setup & Data Loading

In [ ]:
import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Paths — prefer your project path if exists; otherwise use bundled data
USER_ROOT = Path("/Users/hust/bootcamp_zheyu_dong/homework/stage11")
PREFERRED = USER_ROOT / "data" / "data_stage11_eval_risk.csv"
FALLBACK  = Path("/mnt/data/stage11/data/data_stage11_eval_risk.csv")

DATA_CSV = PREFERRED if PREFERRED.exists() else FALLBACK
print("Using dataset:", DATA_CSV)

df = pd.read_csv(DATA_CSV, parse_dates=["date"]).sort_values("date")
df.head()

## 1) Baseline Fit & Metric (time-aware split)

In [ ]:
from sklearn.model_selection import train_test_split

# Ensure time-aware split: last 25% as test
cut = int(len(df) * 0.75)
train_df = df.iloc[:cut].copy()
test_df  = df.iloc[cut:].copy()

features = ["x1","x2"]
target   = "y"

X_train = train_df[features]
y_train = train_df[target]
X_test  = test_df[features]
y_test  = test_df[target]

print("Train/Test sizes:", len(train_df), len(test_df))
X_train.head()

### Pipelines & Scenarios

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Scenario A: Mean imputation + scaling + LinearRegression
pipe_mean = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

# Scenario B: Median imputation + scaling + LinearRegression
pipe_median = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

# Scenario C (optional): PolynomialFeatures (degree=2) + mean impute + scaling + LinearRegression
pipe_poly = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

pipes = { "mean": pipe_mean, "median": pipe_median, "poly2": pipe_poly }

### Fit, Predict, Residuals

In [ ]:
def eval_pipe(name, pipe, X_train, y_train, X_test, y_test):
    pipe.fit(X_train, y_train)
    pred_train = pipe.predict(X_train)
    pred_test  = pipe.predict(X_test)
    rmse_tr = mean_squared_error(y_train, pred_train, squared=False)
    rmse_te = mean_squared_error(y_test, pred_test, squared=False)
    mae_tr  = mean_absolute_error(y_train, pred_train)
    mae_te  = mean_absolute_error(y_test, pred_test)
    res = {
        "name": name, "rmse_train": rmse_tr, "rmse_test": rmse_te,
        "mae_train": mae_tr, "mae_test": mae_te,
        "pred_train": pred_train, "pred_test": pred_test,
        "resid_train": y_train - pred_train, "resid_test": y_test - pred_test
    }
    return res

results = {k: eval_pipe(k, p, X_train, y_train, X_test, y_test) for k, p in pipes.items()}
pd.DataFrame([{k: v for k, v in r.items() if isinstance(v, (int, float))} for r in results.values()])

### Residual Diagnostics (baseline = mean)

In [ ]:
base = results["mean"]
fig, axes = plt.subplots(2, 2, figsize=(10,8))

# Residuals vs fitted (test)
axes[0,0].scatter(base["pred_test"], base["resid_test"], alpha=0.5)
axes[0,0].axhline(0, linewidth=1)
axes[0,0].set_title("Residuals vs Fitted (test)")
axes[0,0].set_xlabel("Fitted")
axes[0,0].set_ylabel("Residuals")

# Histogram of residuals
axes[0,1].hist(base["resid_test"], bins=30)
axes[0,1].set_title("Residual histogram (test)")

# Residuals over time
axes[1,0].plot(test_df["date"], base["resid_test"])
axes[1,0].axhline(0, linewidth=1)
axes[1,0].set_title("Residuals over time (test)")
axes[1,0].set_xlabel("Date")

# True vs Pred
axes[1,1].plot(test_df["date"], y_test.values, label="truth")
axes[1,1].plot(test_df["date"], base["pred_test"], label="prediction")
axes[1,1].legend()
axes[1,1].set_title("Prediction vs Truth (test)")

plt.tight_layout(); plt.show()

## 2) Bootstrap — Uncertainty for RMSE & Prediction Band (baseline = mean)

In [ ]:
from src.evaluation import bootstrap_model_metric, bootstrap_prediction_band, rmse

boots = bootstrap_model_metric(pipes["mean"], X_train, y_train, X_test, y_test, metric_fn=rmse, n_boot=500, random_state=0)
boots_ci = (boots["ci_low"], boots["ci_high"])
print("Bootstrap RMSE 95% CI:", boots_ci)

# Visualize RMSE distribution
plt.figure(figsize=(6,4))
plt.hist(boots["metrics"], bins=30)
plt.axvline(boots_ci[0], linestyle="--")
plt.axvline(boots_ci[1], linestyle="--")
plt.title("Bootstrap RMSE (test)")
plt.xlabel("RMSE")
plt.ylabel("Frequency")
plt.tight_layout(); plt.show()

# Prediction band on test (quantiles per timestamp)
band = bootstrap_prediction_band(pipes["mean"], X_train, y_train, X_test, n_boot=300, random_state=0)
q_lo, q_med, q_hi = band["quantiles"]

plt.figure(figsize=(10,4))
plt.plot(test_df["date"], y_test.values, label="truth")
plt.plot(test_df["date"], q_med, label="pred median")
plt.fill_between(test_df["date"], q_lo, q_hi, alpha=0.3, label="95% band")
plt.title("Prediction with Bootstrap Band (test)")
plt.legend()
plt.tight_layout(); plt.show()

## 3) Scenario Comparison — Mean vs Median Imputation (Linear)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,4), sharey=True)
for ax, key in zip(axes, ["mean","median"]):
    r = results[key]
    ax.plot(test_df["date"], y_test.values, label="truth")
    ax.plot(test_df["date"], r["pred_test"], label="pred")
    ax.set_title(f"{key} imputation — RMSE={r['rmse_test']:.3f}")
    ax.legend()
axes[0].set_ylabel("y")
plt.suptitle("Scenario: Mean vs Median (Linear Regression)", y=1.02)
plt.tight_layout(); plt.show()

## 4) Scenario Comparison — Linear vs PolynomialFeatures (degree=2)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,4), sharey=True)
for ax, key in zip(axes, ["mean","poly2"]):
    r = results[key]
    ax.plot(test_df["date"], y_test.values, label="truth")
    ax.plot(test_df["date"], r["pred_test"], label="pred")
    ax.set_title(f"{key} — RMSE={r['rmse_test']:.3f}")
    ax.legend()
axes[0].set_ylabel("y")
plt.suptitle("Scenario: Linear vs Polynomial (degree=2)", y=1.02)
plt.tight_layout(); plt.show()

## 5) Subgroup Diagnostics — by Segment (A/B/C)

In [ ]:
# Attach predictions from baseline to test_df for subgroup comparison
test_sub = test_df.copy()
test_sub["pred"] = results["mean"]["pred_test"]
test_sub["resid"] = results["mean"]["resid_test"]

# Metrics per segment
rows = []
for seg, g in test_sub.groupby("segment"):
    rows.append({
        "segment": seg,
        "rmse": mean_squared_error(g["y"], g["pred"], squared=False),
        "mae": mean_absolute_error(g["y"], g["pred"]),
        "n": len(g)
    })
seg_table = pd.DataFrame(rows).sort_values("segment")
display(seg_table)

# Visuals: residual box by segment & bar of RMSE
fig, axes = plt.subplots(1,2, figsize=(10,4))

# Boxplot using matplotlib
segments = seg_table["segment"].tolist()
data = [test_sub.loc[test_sub["segment"]==s, "resid"].values for s in segments]
axes[0].boxplot(data, labels=segments)
axes[0].set_title("Residuals by Segment (test)")
axes[0].set_xlabel("segment")
axes[0].set_ylabel("residual")

# Bar chart RMSE
axes[1].bar(seg_table["segment"], seg_table["rmse"])
axes[1].set_title("RMSE by Segment (test)")
axes[1].set_xlabel("segment")
axes[1].set_ylabel("RMSE")

plt.tight_layout(); plt.show()

## 6) Stakeholder Summary (≤ 1 page)

**Assumptions**
- Relationship is approximately linear (or captured by degree=2 terms). 
- Missing values are at random; imputation (mean/median) is appropriate.
- Recent dynamics stable enough that training window generalizes to the last 25% (test).

**Risks**
- Structural breaks/seasonality shifts would hurt accuracy; fixed-window features may go stale.
- Non-random missingness could bias imputation; try robust imputers or model-based imputation.
- Segment C shows higher error (see subgroup); decisions on C should use wider error margins.

**Sensitivity**
- Bootstrap RMSE 95% CI quantifies uncertainty in test performance.
- Mean vs median imputation yields similar trends; choose based on robustness to skew/outliers.
- Polynomial features can reduce bias if mild curvature exists; watch for variance increase.

**Takeaway**
- Baseline is serviceable and reasonably stable within the CI band on test. 
- For production decisions: monitor segment-specific error; re-train if error band widens or drift detected.
- Next: tune regularization, add seasonality/interaction terms, and consider robust models if heteroscedasticity persists.